In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import *
import csv
import seaborn as sns
import xlrd
import ydata_profiling
from pandas_profiling import ProfileReport
import geopandas as gpd
import folium
import mapclassify
import pycountry_convert
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
import geopy
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster

/var/folders/y2/nnpdyztj42sft29s3dp0nybr0000gn/T/ipykernel_62685/534121062.py:9: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [5]:
#First: Read in the kaggle excel dataset
world = pd.read_csv('world-data-2023.csv')

#Create dataframe
world_df = pd.DataFrame(world)

#The Abbrevation, Capital/Major City, Calling Code, and Largest city columns aren't necessary for the purpose of this assignment
world_df = world_df.drop(['Abbreviation','Capital/Major City','Calling Code','Largest city'], axis=1)

#First: Read in the WHR dataset
xlsx = pd.ExcelFile('DataForFigure2.1WHR2023.xls')

#Create dataframe
WHR_df = pd.read_excel(xlsx, 'Sheet1')

#changing name of Country name column to Country so that the two data sets can be merged
WHR_df = WHR_df.rename(columns={'Country name': 'Country'})

#merge kaggle and WHR datasets
merged_df = pd.merge(world_df, WHR_df, on='Country')

#For the purpose of this project, we'll replace the NaN values with the median value
merged_df = merged_df.fillna(merged_df.median())

/var/folders/y2/nnpdyztj42sft29s3dp0nybr0000gn/T/ipykernel_62685/3289433516.py:23: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged_df = merged_df.fillna(merged_df.median())


In [4]:
#creating new df without some columns for time/processing 
merged_df2 = merged_df.drop(['Dystopia + residual','Explained by: Perceptions of corruption','Explained by: Generosity',
                             'Explained by: Freedom to make life choices','Explained by: Healthy life expectancy',
                             'Explained by: Social support','Explained by: Log GDP per capita','Ladder score in Dystopia',
                            'upperwhisker','lowerwhisker','Standard error of ladder score','Ladder score','CPI',
                            'Currency-Code', 'Official language','Latitude','Longitude','Agricultural Land( %)',
                            'Forested Area (%)','GDP',], axis=1)

# I already sent the HTML file for this. It's in markdown here because it takes forever to load

#Merged DF Report
FinalProject_report = ProfileReport(merged_df2, title='2023 World Data')
FinalProject_report.to_file("IST652FinalProject_report.html")

In [11]:
# other df with latitude and longitude

merged_df3 = merged_df.drop(['Dystopia + residual','Explained by: Perceptions of corruption','Explained by: Generosity',
                             'Explained by: Freedom to make life choices','Explained by: Healthy life expectancy',
                             'Explained by: Social support','Explained by: Log GDP per capita','Ladder score in Dystopia',
                            'upperwhisker','lowerwhisker','Standard error of ladder score','Ladder score','CPI',
                            'Currency-Code', 'Official language','Agricultural Land( %)',
                            'Forested Area (%)','GDP',], axis=1)
#function to convert to alpah2 country codes and continents
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)

#function to get longitude and latitude data from country name
geolocator =  Nominatim(user_agent="http")
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan

#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(merged_df3)):
        lat = merged_df3.iloc[i]['Latitude']
        long = merged_df3.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Healthy Life Expectancy : {}<br>"""
        popup_text = popup_text.format(merged_df3.iloc[i]['Country'],
                                   merged_df3.iloc[i]['Healthy life expectancy']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

In [7]:
def get_top_n_results(df, n):
    # Get the top n results sorted highest to lowest by Ladder score and Country
    top_n_results = df.sort_values(by=['Ladder score', 'Country'], ascending=False)[:n]

    # Add a column for the rank
    top_n_results['Rank'] = range(1, n+1)

    # Reset the index and remove the row numbers
    top_n_results = top_n_results.reset_index(drop=True)

    # Format the rank column
    top_n_results['Rank'] = top_n_results['Rank'].astype(str).str.zfill(2)

    # Return the top n results without row numbers as a string
    return top_n_results[['Rank', 'Country', 'Ladder score']].to_string(index=False)

# Usage
top_15_results_str = get_top_n_results(merged_df, 15)
print('Top 15 happiest countries in the world')
print(top_15_results_str)

Top 15 happiest countries in the world
Rank       Country  Ladder score
  01       Finland        7.8042
  02       Denmark        7.5864
  03       Iceland        7.5296
  04        Israel        7.4729
  05   Netherlands        7.4030
  06        Sweden        7.3952
  07        Norway        7.3155
  08   Switzerland        7.2401
  09    Luxembourg        7.2279
  10   New Zealand        7.1229
  11       Austria        7.0973
  12     Australia        7.0946
  13        Canada        6.9607
  14 United States        6.8937
  15       Germany        6.8918


In [10]:
def get_bottom_n_results(df, n):
    # Get the top n results sorted highest to lowest by Ladder score and Country
    top_n_results = df.sort_values(by=['Ladder score', 'Country'], ascending=True)[:n]

    # Add a column for the rank
    top_n_results['Rank'] = range(1, n+1)

    # Reset the index and remove the row numbers
    top_n_results = top_n_results.reset_index(drop=True)

    # Format the rank column
    top_n_results['Rank'] = top_n_results['Rank'].astype(str).str.zfill(2)

    # Return the top n results without row numbers as a string
    return top_n_results[['Rank', 'Country', 'Ladder score']].to_string(index=False)

# Usage
bottom_15_results_str = get_bottom_n_results(merged_df, 15)
print('Bottom 15 happiest countries in the world')
print(bottom_15_results_str)

Bottom 15 happiest countries in the world
Rank      Country  Ladder score
  01  Afghanistan        1.8590
  02      Lebanon        2.3922
  03 Sierra Leone        3.1376
  04     Zimbabwe        3.2035
  05     Botswana        3.4353
  06       Malawi        3.4952
  07      Comoros        3.5452
  08     Tanzania        3.6938
  09       Zambia        3.9822
  10   Madagascar        4.0191
  11        India        4.0358
  12      Liberia        4.0423
  13     Ethiopia        4.0906
  14       Jordan        4.1198
  15         Togo        4.1374
